In [2]:
from astropy.io import fits 
import pandas as pd
import matplotlib.pyplot as plt
#import smplotlib
import numpy as np
import torch 
from spender.data.sdss import BOSS
import os 
from tqdm import tqdm 
import shutil

In [55]:
data_path = "/Users/yaroslav/Documents/2. work/Research/GitHub/AGN-UMAP/data/sdss-qso-catalogue/all-spectra"
mini_batch_dir_train = '/Users/yaroslav/Documents/2. work/Research/GitHub/AGN-UMAP/data/sdss-qso-catalogue/mini-batch/fits/train'
mini_batch_dir_val = '/Users/yaroslav/Documents/2. work/Research/GitHub/AGN-UMAP/data/sdss-qso-catalogue/mini-batch/fits/val'
plates = []
mjds = []
fiberids = []
zs = []

#ids = ((412, 52254, 308), (412, 52250, 129))
for sub_dir_ in tqdm(os.listdir(data_path)): 
    if sub_dir_ != '.DS_Store':
        
        sub_dir = os.path.join(data_path, sub_dir_)
        for file in os.listdir(sub_dir): 
            file_path = os.path.join(sub_dir, file)
            
            if len(zs)==5: 
                break 
            
            elif '.1' not in file and '.DS_' not in file:       
                with fits.open(file_path) as hdulist:           
                    z = hdulist[2].data[0]["Z"]                 
                
                if z <= 1.5 and len(zs)<100:
                    
                    
                    hdulist = fits.open(file_path)
                    spec = hdulist[1].data['flux']
                    wavelength = 10**hdulist[1].data['loglam']

                    if min(wavelength)/(1+z)<2000 and max(wavelength)/(1+z)>2500: 
                        if np.isnan(spec).sum() > 0: 
                            print("NAN?")

                        else: 
                    
                            zs.append(z)
                            
                            if len(zs)<=4:
                                sub_batch_dir = os.path.join(mini_batch_dir_train, sub_dir_)

                            else: 
                                sub_batch_dir = os.path.join(mini_batch_dir_val, sub_dir_)

                            if not os.path.exists(sub_batch_dir): 
                                os.mkdir(sub_batch_dir)

                            batch_file_path = os.path.join(sub_batch_dir, file)

                            shutil.copyfile(file_path, batch_file_path)
                        
                            file_list = file.split('-')
                            plate = int(file_list[1])
                            plates.append(plate)
                            mjd = int(file_list[2])
                            mjds.append(mjd)
                            fiberid = int(file_list[3].split('.')[0])
                            fiberids.append(fiberid)

                            fig, axs = plt.subplots(2, 1, figsize=(6, 4))

                            with fits.open(file_path) as hdulist: 
                                axs[0].plot(10**hdulist[1].data['loglam'], hdulist[1].data['model'])
                                axs[0].set(xlabel='Observed Wavelength (A)', ylabel='Model Flux')
                                
                                axs[1].plot(10**hdulist[1].data['loglam']/(1+z), hdulist[1].data['model'])
                                axs[1].set(xlabel='Rest Wavelength (A)', ylabel='Model Flux', xlim=(1000,6000))
                            
                            fig.tight_layout()
                            plt.savefig(f'/Users/yaroslav/Documents/2. work/Research/GitHub/AGN-UMAP/code/topical/modifying-spender/post-yan/plots/mini-test/{plate}-{mjd}-{fiberid}.png')
                            plt.close()
                    
ids = list((plates[i], mjds[i], fiberids[i]) for i in range(len(plates)))[0:100]

train_ids = ids[0:90]
val_ids = ids[90:]

train_zs = zs[0:90]
val_zs = zs[90:]

100%|██████████| 79/79 [00:02<00:00, 33.79it/s]


## make batches

In [56]:
spec, w, z, norm, zerr = BOSS.make_batch('/Users/yaroslav/Documents/2. work/Research/GitHub/AGN-UMAP/data/sdss-qso-catalogue/mini-batch/fits/train', fields=train_ids) 

spec = np.nan_to_num(spec)

print(np.isnan(spec).sum())

BOSS.save_batch(dir='/Users/yaroslav/Documents/2. work/Research/GitHub/spender/train', batch=[spec,w, z,norm,zerr], tag='train', counter=0)
BOSS.save_batch(dir='/Users/yaroslav/Documents/2. work/Research/GitHub/AGN-UMAP/data/sdss-qso-catalogue/mini-batch/pickle/', batch=[spec,w, z,norm,zerr], tag='train', counter=0)

spec, w, z, norm, zerr = BOSS.make_batch('/Users/yaroslav/Documents/2. work/Research/GitHub/AGN-UMAP/data/sdss-qso-catalogue/mini-batch/fits/val', fields=val_ids) 

print(np.isnan(spec).sum())

BOSS.save_batch(dir='/Users/yaroslav/Documents/2. work/Research/GitHub/spender/train', batch=[spec,w, z,norm,zerr], tag='valid', counter=0)
BOSS.save_batch(dir='/Users/yaroslav/Documents/2. work/Research/GitHub/AGN-UMAP/data/sdss-qso-catalogue/mini-batch/pickle/', batch=[spec,w, z,norm,zerr], tag='valid', counter=0)

#/Users/yaroslav/Documents/2. work/Research/GitHub/AGN-UMAP/data/sdss-qso-catalogue/mini-batch/pickle/

downloading https://data.sdss.org/sas/dr16/eboss/spectro/redux/v5_13_0/spectra/lite//7167/spec-7167-56604-0919.fits
0
tensor(0)


In [44]:
train_batch_path = '/Users/yaroslav/Documents/2. work/Research/GitHub/spender/train/BOSStrain_0.pkl'
valid_batch_path = '/Users/yaroslav/Documents/2. work/Research/GitHub/spender/train/BOSSvalid_0.pkl'

In [54]:
import pickle 

with open(train_batch_path, "rb") as f:
    batch = pickle.load(f)
    print(len(batch), [len(i) for i in batch], [len(j) for j in batch[0]])

with open(valid_batch_path, "rb") as f:
    batch = pickle.load(f)
    print(len(batch), [len(i) for i in batch], [len(j) for j in batch[0]])

5 [90, 90, 90, 90, 90] [4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686]
5 [10, 10, 10, 10, 10] [4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686, 4686]
